In [1]:
%pip install tensorflow

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [85]:
# read the data
#-------------------
data = pd.read_csv("final_data.csv")
data.describe()
data = data[data["mag"] >= 3]

Training Data

In [86]:
from sklearn.model_selection import train_test_split

X = data[["latitude", "longitude", "timestamp"]]
y = data[["mag", "depth"]]
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(664875, 3) (664875, 2)
(531900, 3) (531900, 2) (132975, 3) (132975, 2)


In [87]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)

In [88]:
def get_basic_model():
	model = tf.keras.Sequential([
		normalizer,
		tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
		tf.keras.layers.Dense(16, activation='relu'),
		tf.keras.layers.Dense(2, activation='softmax')
	])
	model.compile(optimizer='adam',
			loss=tf.keras.losses.SquaredHinge(),
			metrics=['accuracy'])
	return model

In [89]:
model = get_basic_model()
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
16622/16622 [==============================] - 42s 2ms/step - loss: 0.0061 - accuracy: 0.9648 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 2/15
16622/16622 [==============================] - 104s 6ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 3/15
16622/16622 [==============================] - 171s 10ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 4/15
16622/16622 [==============================] - 187s 11ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 5/15
16622/16622 [==============================] - 138s 8ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 6/15
16622/16622 [==============================] - 157s 9ms/step - loss: 0.0061 - accuracy: 0.9667 - val_loss: 0.0062 - val_accuracy: 0.9667
Epoch 7/15
16622/16622 [==============================] - 136s 8ms/step - loss: 0.0061 - accuracy: 0.9667 - val

In [90]:
history.history

{'loss': [0.0060842460952699184,
  0.0060734618455171585,
  0.006072208285331726,
  0.006071118637919426,
  0.006070575676858425,
  0.00607006810605526,
  0.006069852504879236,
  0.0060693444684147835,
  0.006069249007850885,
  0.006069120019674301,
  0.006069410592317581,
  0.006068889517337084,
  0.006068604066967964,
  0.00606864457949996,
  0.00606816029176116],
 'accuracy': [0.9647809863090515,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666779637336731,
  0.9666685461997986,
  0.9666516184806824,
  0.9666760563850403,
  0.9666760563850403,
  0.9666779637336731,
  0.9666704535484314,
  0.9666779637336731],
 'val_loss': [0.0062282090075314045,
  0.006224975921213627,
  0.006227517034858465,
  0.0062241763807833195,
  0.006224492564797401,
  0.006224878132343292,
  0.00622567068785429,
  0.006224952172487974,
  0.0062220292165875435,
  0.006222139578312635,
  0.006222588010132313,
  0.006226

In [92]:
%pip install pyyaml h5py


   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   -- ------------------------------------- 10.2/144.7 kB ? eta -:--:--
   ---------------------------------------  143.4/144.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 144.7/144.7 kB 1.4 MB/s eta 0:00:00


In [96]:

# Save the entire model as a `.keras` zip archive.
model.save('neural_network.tf')

INFO:tensorflow:Assets written to: neural_network.tf\assets


INFO:tensorflow:Assets written to: neural_network.tf\assets


In [97]:
new_model = tf.keras.models.load_model('neural_network.tf')

In [98]:
new_model.evaluate(X_test[0:1000], y_test[0:1000], verbose=2)

32/32 - 0s - loss: 0.0044 - accuracy: 0.9680 - 276ms/epoch - 9ms/step


[0.004406984895467758, 0.9679999947547913]